<a href="https://colab.research.google.com/github/ben14132/ITN-finetuning/blob/main/newllamafinetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install  -q ipywidgets
!pip install -q scipy
!pip install sentencepiece


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-u3wt4b8_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-u3wt4b8_
  Resolved https://github.com/huggingface/peft.git to commit ffb8512396e24e9e60e9f1fec02052ac6b407a23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.10.1.dev0-py3-none-any.whl size=199142 sha256=b167f6e99a4c9df0b2548732c6df1248ce312d50576b8abed14ea570a3d424ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-xd_jh85r/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.10.0
    Uninstalling peft-0.10.0:
      Successfully uninstalled peft-0.10.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ree3dtb1
  Running command gi

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import sentencepiece
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig, pipeline
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
from datasets import load_dataset,Features,Value
train_dataset = load_dataset('json', data_files='datafile7.json', split='train')
train_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 3100
})

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,use_fast=False,add_eos_token=True) # Set trust_remote_code=True
tokenizer.pad_token_id = 18610


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
print(tokenizer.decode(tokenizer.encode("hello"), add_special_tokens = False))

<s> hello</s>


In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
)

peft_model = get_peft_model(model, peft_config)

In [ ]:

output_dir = "/content/gdrive/MyDrive/LLM24"
per_device_train_batch_size = 2 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 2  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 120        # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    bf16=False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    tf32=False,
)


In [ ]:
def formatting_func(sample):
    output_text = []
    for i in range(len(sample['instruction'])):
        text = f"""
        Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction: {sample['instruction'][i]}

        ### Input:
        {sample['input'][i]}

        ### Response:
        {sample['output'][i]}"""
        output_text.append(text)
    return output_text



In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    max_seq_length=1024,
    formatting_func=formatting_func,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
peft_model.config.use_cache = False
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.090800
20,0.810300
30,0.385200
40,0.274300
50,0.331700
60,0.506100
70,0.330600
80,0.259800
90,0.223100
100,0.334500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=120, training_loss=0.5265778779983521, metrics={'train_runtime': 988.787, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.121, 'total_flos': 1888192394919936.0, 'train_loss': 0.5265778779983521, 'epoch': 0.15})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Ben141/LLM24/commit/d98515e2426db797d49bc91b52f46f9ba0ca1e7c', commit_message='End of training', commit_description='', oid='d98515e2426db797d49bc91b52f46f9ba0ca1e7c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
PEFT_MODEL = "Ben141/LLM24"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# peft_base_model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     return_dict=True,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)




adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

In [ ]:
# Loading original model

model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_answer(query):
  user_prompt = f"""{query}"""

  final_prompt = f"""
  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
  ### Instruction:
  The following sentence is in its spoken form, convert it to its written form.

  ### Input:
  {user_prompt}

  ### Response:"""


  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  # encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  # outputs = model.generate(input_ids=encoding.input_ids,  generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id,\
  #                                                                                                                    eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
  #                                                                                                                    temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  # text_output = tokenizer.decode(outputs[0], skip_special_tokens = True)

  # print(dashline)
  # print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id,  \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens = True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

In [ ]:
generate_answer('')

-------------------------------------------------
PEFT MODEL RESPONSE:

  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
  ### Instruction:
  The following sentence is in its spoken form, convert it to its written form.

  ### Input:
  my shoe size is eleven and a half

  ### Response:
  My shoe size is 11½
-------------------------------------------------
